<a href="https://colab.research.google.com/github/Trifasick0/generadorPoesia/blob/main/Generador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers
!apt install git-lfs
!pip install syltippy #Utilizado para separar sílabas para la métrica
!pip install spacy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 30.0 MB/s 
     |████████████████████████████████| 5.3 MB 66.9 MB/s 
     |████████████████████████████████| 163 kB 74.2 MB/s 
     |████████████████████████████████| 212 kB 74.8 MB/s 
     |████████████████████████████████| 115 kB 74.6 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
     |████████████████████████████████| 7.6 MB 55.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not up

In [3]:
import os
import math
import torch
import torchvision
from google.colab import drive
import transformers
from transformers import GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from syltippy import syllabize
import itertools
import re
import spacy.cli
spacy.cli.download("es_core_news_sm")
import es_core_news_sm
nlp = es_core_news_sm.load()
#drive.mount('/content/drive')
#dirname = '/content/drive/MyDrive/TFG_poesia/'
#path = os.chdir(dirname)

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


# Cálculo de la métrica de un verso




In [4]:
vocales = ['a','e','i','o','u','á','é','í','ó','ú']
vocalesAbiertasHiato = ['a','e','o','í','ú']
vocalesCerradasHiato = ['i','u']
tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}

"""
Método que devolverá un resultado dependiendo de si la palabra que le pasamos es
aguda = 1 (ya que se le suma 1 a la métrica si la última palabra es aguda)
llana = 0 (no se cambia la métrica si la última palabra es llana)
esdrújula = -1 (se resta 1 a la métrica si la última palabra es esdrújula)
"""
def palabraTonica(tuplaPalabra):
  resultado = 0
  if tuplaPalabra[1] == (len(tuplaPalabra[0]) - 1): #Palabra aguda, ya que silaba tonica es la ultima 
    resultado = 1
  elif tuplaPalabra[1] == (len(tuplaPalabra[0]) - 2): #Palabra llana, ya que silaba tonica es la antepenultima
    resultado = 0
  elif tuplaPalabra[1] <= (len(tuplaPalabra[0]) - 3): #Esdrujula
    resultado = -1
    
  return resultado

"""
Comprobará si existe sinalefa entre palabraAnterior y palabra, la sinalefa es un
recurso propio de la poesía en el que dos sílabas se unen si la anterior termina
en vocal y la siguiente empieza por vocal
"""
def sinalefa(palabraAnterior, palabra, tuplaPalabra):
  if palabraAnterior.isspace() == False and palabra.isspace() == False:
    #print("POSIBLESINALEFAENTRE: ", palabraAnterior, palabra)
    ultLetra = palabraAnterior[len(palabraAnterior) - 1]
    priLetra = palabra[0]
    resultado = 0
    if ultLetra.lower() in vocales and priLetra.lower() in vocales:
      #print("sinalefa: ", ultLetra, priLetra)
      resultado = -1

    return resultado
"""
Método que devolverá la métrica de verso, comprobando la existencia de sinalefas y
palabras tónicas
"""
def metricaVerso(verso):
  if verso.isspace() == False:
    palabrasVerso = verso.split(" ")
    #print("Verso spliteado", palabrasVerso)
    versoSeparadoSilabas = [syllabize(p) for p in palabrasVerso] #El método syllabize devuelve una tupla con las silabas del texto y la posicion de la silaba tonica del mismo
    numSilabas = 0

    for p in range(len(versoSeparadoSilabas)):
      tupla = versoSeparadoSilabas[p]

      if p > 0 : #Comprobamos que no exista sinalefa (ultima silaba de la palabra anterior se une con la primera de la siguiente palabra)
        numSilabas += sinalefa(palabrasVerso[p-1], palabrasVerso[p], tupla)

      if p == (len(versoSeparadoSilabas) - 1) and len(tupla[0]) > 1: #Comprobamos si la ultima palabra es aguda (+1) llana (+0) o esdrujula (-1)
        numSilabas += palabraTonica(tupla)                           #Tenemos en cuenta que los monosílabos siguen sus propias reglas de acentuación
                                                                     #Para que una palabra sea aguda llana o esdrújula deben tener más de 1 sílaba
      numSilabas += len(tupla[0])

    return numSilabas
  else:
    return 0

print(metricaVerso("flores y la vida son cuatro bueyes y"))

12


# Detección de rimas

In [5]:
"""
Método que elimina tildes y espacios en blanco de la palabra p
"""
def normalizar(p):
  resultado = p.lower()
  for t in tildes:
	  if t in resultado:
		  resultado = resultado.replace(t, tildes[t])
  if ' ' in resultado:
    resultado = resultado.replace(' ', '')
  return resultado
"""
Recoge la vocal tónica de una palabra p, esta vocal tónica es la vocal que le 
corresponde a la sílaba tónica (cada sílaba tiene una vocal)
"""
def getVocalTonica(p):
  silabas = syllabize(p)
  silabaTonica = silabas[0][silabas[1]]
  posVocalTonica = 0

  for pos in range(len(silabaTonica)):

    if silabaTonica[pos] in vocalesCerradasHiato: #Posible diptongo: Las vocales cerradas de los diptongos no cuentan en la rima
      if (pos + 1) in range(len(silabaTonica)): 
        if silabaTonica[pos + 1] not in vocalesAbiertasHiato:
          posVocalTonica = pos
          

      elif (pos - 1) in range(len(silabaTonica)): #Posible diptongo: Las vocales cerradas de los diptongos no cuentan en la rima
        if silabaTonica[pos] in vocalesCerradasHiato and silabaTonica[pos - 1] not in vocalesAbiertasHiato:
          posVocalTonica = pos
          

    elif silabaTonica[pos] in vocales: #Caso general
      posVocalTonica = pos
      

  if silabas[1] != 0: #Devolvemos la posición de la vocal tónica dentro de la palabra
    for posSilaba in range(0, silabas[1]):
      posVocalTonica += len(silabas[0][posSilaba])    
  return posVocalTonica
"""
Recoge los fonemas de una palabra pero en un caso especial, en las palabras 
esdrújulas existe el caso de que por conveniencia se elimine la sílaba 
postónica, esto se hace para conseguir rima. Ejemplo:
zanco y cántico 
no riman normalmente, pero si eliminamos -ti- de cántico
rimará de forma consonante (anco)
"""
def fonemaPalabraEsdrujula(p, posVocalTonica):
  s1 = syllabize(p)
  p1Aux = p
  if palabraTonica(s1) == -1: #Esdrújula
    p1Aux = ""
    for pos in range(0, len(s1[0])):
      if pos != (s1[1] + 1): #Si no es la postónica
        p1Aux += s1[0][pos]
  left, resultado = p1Aux[:posVocalTonica], p1Aux[posVocalTonica:]
  return resultado
"""
Es otro caso especial de la rima en el que se elimina la vocal débil de un diptongo
(la vocal cerrada, es decir [i,u])
"""
def eliminaVocalesCerradasDiptongos(p):
  resultado = ""

  for pos in range(0, len(p)):
    if p[pos] not in vocalesCerradasHiato: #Si la letra no es vocal cerrada
      resultado += p[pos]
    else: #Si la vocal es cerrada
      vocalAnteriorAbierta = False
      vocalPosteriorAbierta = False
      if (pos - 1) in range(0, len(p)) and p[pos-1] in vocalesAbiertasHiato: #Si hace diptongo con el anterior y es abierta
        vocalAnteriorAbierta = True
      if (pos + 1) in range(0, len(p)) and p[pos+1] in vocalesAbiertasHiato: #Si hace diptongo con el posterior y es abierta
        vocalPosteriorAbierta = True
      if vocalAnteriorAbierta == False and vocalPosteriorAbierta == False: #No hace diptongo con abierta ni anterior ni posterior
        resultado += p[pos]

  return resultado
"""
Recogerá el fonema de la palabra
"""
def fonemaPalabra(p, posVocalTonica):
  s1 = syllabize(p)
  left, resultado = p[:posVocalTonica], p[posVocalTonica:]
  return resultado
"""
Compara dos fonemas de forma consonante
"""
def compararFonemasRimaConsonante(f1,f2):
  normF1 = normalizar(f1)
  normF2 = normalizar(f2)
  if normF1 == normF2:
    return normF1
  else:
    return None
"""
Compara dos fonemas de forma asonante
"""
def compararFonemasRimaAsonante(f1,f2):
  vocalesF1 = ""
  vocalesF2 = ""
  for l in f1:
    if l in vocales:
      vocalesF1 += l
  for l in f2:
    if l in vocales:
      vocalesF2 += l
  normVocalesF1 = normalizar(vocalesF1)
  normVocalesF2 = normalizar(vocalesF2)
  if normVocalesF1 == normVocalesF2:
    return normVocalesF1
  else:
    return None
"""
Comparará fonemas llamando a los métodos anteriores (asonante y consonante)
"""
def comparaFonemas(f1,f2):
  resultado = compararFonemasRimaConsonante(f1,f2)
  if resultado == None:
    resultado = compararFonemasRimaAsonante(f1,f2)
  return resultado
"""
Método que detectará la rima entre dos palabras (si existe) utilizando los métodos
anteriormente descritos
"""
def rimaPalabras(p1, p2):
  #print("RIMA ENTRE P1 ", p1, "Y P2", p2)
  #Necesitamos las posiciones de las vocales tónicas de cada palabra
  posVocalTonicaP1 = getVocalTonica(p1)
  posVocalTonicaP2 = getVocalTonica(p2)
  
  #Ahora detectaremos la rima, que es la repetición de letras a partir de la vocal tónica
  #Para ello cogeremos las partes de ambas palabras que pueden rimar (a partir de la vocal tónica)
  #Hay que tener en cuenta un caso especial de rima en el que si la palabra es esdrújula, se elimina la sílaba postónica (Aunque no se elimina siempre, solo si es necesario para la composicion del poema)
  #Hay que tener en cuenta otro caso especial de rima en el que las vocales cerradas de los diptongos no se tienen en cuenta (Aunque tampoco se elimina siempre como en el anterior caso)
  #Para cubrir todas las posibilidades, crearemos dos set para cada posibilidad de cada palabra, posteriormente mediante un producto cartesiano de ambos sets tendremos todas las posibles combinaciones de fonemas
  setP1 = set()
  setP2 = set()

  #Fonema normal
  fonP1 = fonemaPalabra(p1, posVocalTonicaP1)
  setP1.add(fonP1)
  fonP2 = fonemaPalabra(p2, posVocalTonicaP2)
  setP2.add(fonP2)
  #Fonema con esdrujulas (Eliminando postónica, si las hay)
  fonEsdrujulaP1 = fonemaPalabraEsdrujula(p1, posVocalTonicaP1)
  setP1.add(fonEsdrujulaP1)
  fonEsdrujulaP2 = fonemaPalabraEsdrujula(p2, posVocalTonicaP2)
  setP2.add(fonEsdrujulaP2)
  #Fonema normal eliminando vocales cerradas en diptongos
  fonP1Diptongo = eliminaVocalesCerradasDiptongos(fonP1)
  setP1.add(fonP1Diptongo)
  fonP2Diptongo = eliminaVocalesCerradasDiptongos(fonP2)
  setP2.add(fonP2Diptongo)
  #Fonema con esdrujulas (Eliminando postónica, si las hay) y eliminando vocales cerradas en diptongos
  fonEsdrujulaP1Diptongo = eliminaVocalesCerradasDiptongos(fonEsdrujulaP1)
  setP1.add(fonEsdrujulaP1Diptongo)
  fonEsdrujulaP2Diptongo = eliminaVocalesCerradasDiptongos(fonEsdrujulaP2)
  setP2.add(fonEsdrujulaP2Diptongo)

  #print("Set fonemas p1:", setP1)
  #print("Set fonemas p2:", setP2)
  #Calculamos todas las posibles combinaciones de fonemas
  prodCartesiano = list(itertools.product(list(setP1),list(setP2))) #Utilizamos el producto cartesiano para obtener todas las combinaciones posibles de ambos sets
  #print("Producto cartesiano:", prodCartesiano)
  
  for c in prodCartesiano:
    r = comparaFonemas(c[0],c[1])
    if r != None:
      return r


  return None
  

p1 = "cántico"
p2 = "zanco"
r = rimaPalabras(p1, p2)
print(r)

anco


# Carga del modelo

In [6]:
model_checkpoint = "PlanTL-GOB-ES/gpt2-large-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
localModel = GPT2LMHeadModel.from_pretrained("ss000045/gpt2-large-bne-poesiaHispanica")

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

# Funciones auxiliares para el generador


In [7]:
#Método utilizado para juntar strings
def juntaStrings(str1, str2):
  str1_strip = str1.strip()
  str2_strip = str2.strip()
  resultado = ""
  if str1 == "":
    resultado = str2_strip
  elif str2 == "":
    resultado = str1_strip
  else:
    resultado = str1_strip + " " + str2_strip
  return resultado

#Recogerá del texto las palabras necesarias para alcanzar la métrica objetivo
def getTextoMetrica(texto, metricaObjetivo):
  palabras = texto.split(" ")
  resultado = ""
  metricaResultado = 0
  resultadoAux = ""
  for p in palabras:
    resultadoAux = juntaStrings(resultadoAux, p)
    #print("Calcularemos la metrica de: ", resultadoAux)
    metricaAux = metricaVerso(resultadoAux)
    if metricaAux <= metricaObjetivo:
      resultado = resultadoAux
      metricaResultado = metricaAux
  return (resultado, metricaResultado)

# Constantes para el generador
Modificar según sea necesario, se explicarán todas las variables:
- LIMITE_GENERACIONES: El número máximo de veces que se generarán posibles versos en un *NodoGeneracion*

- NUM_TEXTOS_GENERACION: El número de salidas de posibles versos que se generarán en una generación.

- MIN_TOKENS_A_GENERAR: Mínimo de tokens que generará el generador

- MAX_TOKENS_A_GENERAR: Máximo de tokens que generará el generador

- CORTA_MITAD: Si es *True* a la hora de comprobar si un posible verso rima y no
lo hace, se cortará este mismo a la mitad de la métrica y se introducirá como
nodo en la pila

- CONST_TOP_P: Es una variable dentro de la generación de texto, es el mínimo de probabilidad acumulada que tiene que tener un set de palabras para ser considerado como salida

- CONST_MIN_TOP_P: Dentro del código, si el generador vuelve a pasos anteriores a menudo (marcado por MAX_NUM_PASOS_ANTERIORES), se restará -0.05 a TOP_P, este es el límite inferior de TOP_P del cual no bajará nunca.

- MAX_NUM_PASOS_ANTERIORES: El número de pasos anteriores seguidos a partir del cual bajaremos TOP_P.

- TOP_K: Es una variable dentro de la generación de texto, es el número de sets de palabras que se escogen como posibles salidas (los k mejores) puede usarse en combinación con TOP_P, si no se quiere utilizar se puede poner a 0


In [14]:
LIMITE_GENERACIONES = 8
NUM_TEXTOS_GENERACION = 15
MIN_TOKENS_A_GENERAR = 20
MAX_TOKENS_A_GENERAR = 25
CORTA_MITAD = False
CONST_TOP_P = 0.85
CONST_MIN_TOP_P = 0.85
MAX_NUM_PASOS_ANTERIORES = 3
TOP_K = 50

# Clase NodoGeneracion
Esta clase representa un paso dentro de la generación de los poemas

In [18]:
class NodoGeneracion:

  def __init__(self, textoEntrada, numVerso, cortaMitad, nodoInicial):
    self.textoEntrada = textoEntrada #Texto de entrada para la generación de textos
    self.numVerso = numVerso #Verso que se está generando
    self.arrayGeneracion = [] #Array de textos que se han generado en esa generación
    self.posGeneracionElegida = -1 #Posiciones dentro del array del verso elegido
    self.numGeneraciones = 0 #Número de generaciones de posibles textos que se han generado
    self.resultado = "" #Resultado
    self.cortaMitad = cortaMitad #Si este nodo se puede cortar a mitad
    self.nodoInicial = nodoInicial #Si este nodo es el inicial (el número de generaciones no tiene límite)
  
  def setNumGeneraciones(self, numGeneraciones):
    self.numGeneraciones = numGeneraciones

  def getTextoEntrada(self):
    return self.textoEntrada
  
  def getCortaMitad(self):
    return self.cortaMitad

  def getNumVerso(self):
    return self.numVerso

  def getGeneracionElegida(self):
    if self.posGeneracionElegida >= 0:
      return self.posGeneracionElegida
    else:
      return -1

  def setPosArrayGeneracion(self, pos, text):
    self.arrayGeneracion[pos] = text

  def eliminaResultado(self):
    self.resultado = ""

  def getResultado(self):
    return self.resultado

  def setResultado(self, resultado):
    self.resultado = resultado

  def setGeneracionElegida(self, posGeneracionElegida):
    if posGeneracionElegida >= 0:
      self.posGeneracionElegida = posGeneracionElegida
      self.resultado = self.arrayGeneracion[self.posGeneracionElegida]

  def getArrayGeneracion(self):
    return self.arrayGeneracion
  """Método que generará textos en el arrayGeneracion (si no se ha llegado al límite)"""
  def generarTexto(self, metricaObjetivo, metricaRestante, textoVerso, TOP_P):
    self.numGeneraciones += 1
    if self.numGeneraciones <= LIMITE_GENERACIONES or self.nodoInicial == True: #Si no se ha llegado al límite de generaciones
      print("GENERACION:", self.numGeneraciones)
      self.arrayGeneracion = []
      self.posGeneracionElegida = -1
      input = self.textoEntrada.replace(',','')
      input_ids = tokenizer.encode(input, add_special_tokens = True ,return_tensors = 'pt') #Tokenizamos la entrada
      #Ahora generamos los posibles versos
      outputs = localModel.generate(input_ids, do_sample = True, top_k = TOP_K, top_p = TOP_P, min_length = (len(input_ids[0]) + MIN_TOKENS_A_GENERAR), max_length = (len(input_ids[0]) + MAX_TOKENS_A_GENERAR), no_repeat_ngram_size = 2, num_return_sequences = NUM_TEXTOS_GENERACION, early_stopping = True)
      for i, output in enumerate(outputs):
        outputDecoded = tokenizer.decode(output, lean_up_tokenization_spaces = True, skip_special_tokens = True)
        entrada, salida = outputDecoded[:len(input)], outputDecoded[len(input):]
        salida = re.sub(' +', ' ', salida).strip() #Eliminamos espacios en blanco sobrantes
        salidaMetrica, metricaSalida = getTextoMetrica(salida, metricaRestante) #Cortamos el texto generado hasta superar la métrica
        aux = juntaStrings(textoVerso, salidaMetrica)
        metricaAux = metricaVerso(aux) #Calculamos la métrica del texto generado junto al texto ya existente en ese verso
        print("posible texto generado", salidaMetrica, "-->mAux:", metricaAux, "-->mObj:", metricaObjetivo)
        if metricaAux == metricaObjetivo: #Si la métrica es igual
          #print("posible texto generado", salidaMetrica, "-->mAux:", metricaAux, "-->mObj:", metricaObjetivo)
          self.arrayGeneracion.append(salidaMetrica.strip()) #Se añade como posible salida
      return True
    else: 
      return False

# Clase GeneradorPoema

In [19]:
class GeneradorPoema:
  #Devolverá el verso que queramos
  def getVerso(self, numVerso):
    resultado = ""
    for i in self.pilaPoema:
      if i.getNumVerso() == numVerso: 
        resultado = juntaStrings(resultado, i.getResultado())
    return resultado


  #Devolverá todos los versos generados hasta el momento
  def getVersos(self):
    resultado = ""
    for i in self.pilaPoema:
      resultado = juntaStrings(resultado, i.getResultado())
    return resultado

  #Pedirá una entrada de texto para generar el poema a partir de la misma entrada
  def pedirEntrada(self):
    print("""Ahora debe introducir una entrada a partir de la cual generar el resto del poema""")
    entrada = input("Introduzca una entrada: ")
    self.entrada = re.sub(' +', ' ', entrada)

  #Pedirá la estructura del poema que se desee generar al usuario
  def pedirEstructura(self):
    self.estructura = []
    print("""Primero de todo, deberá introducir una estructura, se mostrará el formato con un ejemplo: 
-     6a/6b/ /7a/7b/ /5-/6a
cada número seguido de una letra es un verso, los espacios significan cambio de estrofa y el guión (-) significa que ese verso no tiene rima
el mínimo silábico es 2 y el máximo 14""")
    entrada = input("Introduzca una estructura: ")
    arrayEntrada = entrada.split("/")
    print(arrayEntrada)
    for a in arrayEntrada:
      numSilabas = None
      rima = None
      if len(a) == 1 and a[0] == ' ':
        self.estructura.append((0,' '))
      else:
        if len(a) == 2:
          n = int(a[0])
          if n >= 2: numSilabas = n
          if a[1].isalpha() or a[1] == '-': rima = a[1]
        elif len(a) == 3:
          n = int((a[0]+a[1]))
          if n >= 2 and n <= 14: numSilabas = n
          if a[2].isalpha() or a[2] == '-': rima = a[2]

        if numSilabas == None or rima == None: 
          print("Mala estructura:", a)
          self.estructura = []
          break
        else: self.estructura.append((numSilabas, rima))
    #Creamos el diccionario de rimas, almacenará la rima de cada tipo de rima del poema
    self.diccionarioRimas = {}
    for n in self.estructura:
      if n[1] != " ":
        self.diccionarioRimas[n[1]] = ""
    print("DiccionarioRimas:", self.diccionarioRimas)
    print(self.estructura)

  #Si el verso es el último de su párrafo
  def isUltimoVersoParrafo(self, numVerso):
    if numVerso == (len(self.estructura) - 1) or self.estructura[numVerso + 1][0] == 0:
      #print("El verso", numVerso, "es el último de su párrafo")
      return True
    return False
   
  #Devolverá el numero de verso con el que debe rimar el verso que pasamos como parámetro
  def getRimaNumVerso(self, numVerso):
    rima = self.estructura[numVerso][1]
    resultado = None
    if rima != '-':
      for i in range(0, numVerso):
        rima_i = self.estructura[i][1]
        if i != numVerso and rima == rima_i:
          resultado = i
    return resultado

  #Devolverá la rima entre dos versos
  def getRimaVersos(self, verso1, verso2):
    verso1_split = verso1.split(" ")
    verso2_split = verso2.split(" ")
    ultPalabraVerso1 = verso1_split[len(verso1_split) - 1]
    ultPalabraVerso2 = verso2_split[len(verso2_split) - 1]
    if ultPalabraVerso1.lower() != ultPalabraVerso2.lower():
      resultado = rimaPalabras(ultPalabraVerso1, ultPalabraVerso2)
      return resultado
    else:
      print(ultPalabraVerso1, "y", ultPalabraVerso2, "son iguales, no sirve como rima")
      return None
  #Buscará el mejor resultado de un array de textos generados
  def buscaGeneracion(self, posInicio, arrayGeneracion, numVerso, cortaMitad):
    metricaObjetivo = self.estructura[numVerso][0]
    metricaRestante = (metricaObjetivo - metricaVerso(self.getVerso(numVerso)))
    verso = self.getVerso(numVerso)

    if len(arrayGeneracion) > 0:
      for pos in range(posInicio, len(arrayGeneracion)):
        textoGenerado = arrayGeneracion[pos]
        if textoGenerado != "": #Si el generador no ha generado ningun token
          nuevoVerso = juntaStrings(verso, textoGenerado)
          if metricaVerso(nuevoVerso) < metricaObjetivo:
            #print("Nuevo verso:", nuevoVerso, "con métrica menor al objetivo")
            return (pos, 0)
          elif metricaVerso(nuevoVerso) == metricaObjetivo:
            #Debemos comprobar si este verso es el último del párrafo para evitar que termine en preposición, conjunción o determinante
            spacyNuevoVerso = nlp(nuevoVerso)
            spacyUltPalabra = spacyNuevoVerso[len(spacyNuevoVerso) - 1]
            generacionValida = True
            #Reglas de estilo
            if (self.isUltimoVersoParrafo(numVerso) == True or self.estructura[numVerso][1] != '-') and spacyUltPalabra.pos_ not in ("NOUN", "ADJ", "ADV", "VERB") or spacyUltPalabra.lemma_.lower() == "no":
              #Si no se cumplen
              generacionValida = False
              #print("Ult verso del parrafo con preposicion al final:", nuevoVerso)

            if generacionValida == True:
              #Si se cumple, pasaremos a comprobar la rima
              numVersoRima = self.getRimaNumVerso(numVerso)
              if numVersoRima != None:
                #Aqui mirariamos lo de la rima
                resultado = self.getRimaVersos(self.getVerso(numVersoRima), nuevoVerso)
                #Comprobaremos que esta rima coincida con las demás de su mismo tipo
                tipoRima = self.estructura[numVerso][1]
                if self.diccionarioRimas[tipoRima] != "" and resultado != self.diccionarioRimas[tipoRima]:
                  #La rima no es válida
                  resultado = None
                #Si falla:
                if resultado == None:
                  generacionValida = False
                  if cortaMitad == True:
                    #Cortamos el nodo por la mitad
                    return (pos, 2)
              if generacionValida == True:
                #Nodo válido
                print("Resultado:", arrayGeneracion[pos])
                return (pos, 1)


    return None

  def anteriorPaso(self):
    if self.contadorPasosAtras < MAX_NUM_PASOS_ANTERIORES:
      print("Anterior paso")
      nodo = self.pilaPoema[len(self.pilaPoema) - 1]
      numVerso = nodo.getNumVerso()
      metricaObjetivo = self.estructura[numVerso][0]
      #Ahora comprobaremos si en el nodo actual hay otro posible resultado 
      #Comprobamos el array
      print("Probamos con el array ya generado")
      arrayGeneracion = nodo.getArrayGeneracion()
      print("ARRAY: ", arrayGeneracion) 
      print("POSELEGIDA: ", nodo.getGeneracionElegida())
      if len(arrayGeneracion) > 0:
        resultado = self.buscaGeneracion((nodo.getGeneracionElegida() + 1), arrayGeneracion, numVerso, nodo.getCortaMitad())
      else:
        print("No hay array generado")
        resultado = None
      if resultado != None:
        #Se ha conseguido otro resultado en el mismo array
        nodo.setGeneracionElegida(resultado[0])
        return (nodo, resultado[1])
      else:
        #No se ha conseguido un resultado, se seguirá generando
        nodo.setResultado("")
        self.actualizarDiccionarioRimas()
        #print("Estado de la pila del poema: ")
        #[print(i, self.pilaPoema[i].getResultado(), "-->numVerso:",self.pilaPoema[i].getNumVerso()) for i in range(0, len(self.pilaPoema))]
        verso = self.getVerso(numVerso)
        metricaRestante = metricaObjetivo - metricaVerso(verso)
        print("No hay nada en el array ya generado, probamos a generar otro texto")
        #Significa que debemos seguir generando (si se puede) en este nodo
        resultadoConseguido = False
        while resultadoConseguido == False:
          if nodo.generarTexto(metricaObjetivo, metricaRestante, verso, self.TOP_P) == True: #Generamos de nuevo
            arrayGeneracion = nodo.getArrayGeneracion()
            resultado = self.buscaGeneracion(0, arrayGeneracion, numVerso, nodo.getCortaMitad()) #Buscamos en el array generado
            if resultado != None:
              resultadoConseguido = True
              nodo.setGeneracionElegida(resultado[0])
              return(nodo, resultado[1])
          else:
            return None     
    elif self.TOP_P > self.MIN_TOP_P:
      print("MAXIMO DE CONTADOR DE PASOS ATRAS")
      self.TOP_P = self.TOP_P - 0.05
      print("NUEVO TOP_P:", self.TOP_P)
      self.contadorPasosAtras = 0

  #Generará el siguiente nodo en la pila del poema e intentará generar una continuación del texto de entrada que cumpla
  # con la métrica y la rima
  def siguientePaso(self, textoEntrada, numVerso, cortaMitad):
    print("Siguiente paso")
    metricaObjetivo = self.estructura[numVerso][0]
    verso = self.getVerso(numVerso)
    metricaRestante = (metricaObjetivo - metricaVerso(verso))
    if metricaRestante == 1:
      return (None, None)
    nodo = NodoGeneracion(textoEntrada, numVerso, cortaMitad, False)
    resultadoConseguido = False

    while resultadoConseguido == False:
      if nodo.generarTexto(metricaObjetivo, metricaRestante, verso, self.TOP_P) == True: #Generamos texto
        arrayGeneracion = nodo.getArrayGeneracion()
        resultado = self.buscaGeneracion(0, arrayGeneracion, numVerso, nodo.getCortaMitad()) #Buscamos en el array generado
        if resultado != None:
          resultadoConseguido = True
          nodo.setGeneracionElegida(resultado[0])
          return (nodo, resultado[1])
      else: 
        return (None, None)
  
  #Devolverá los versos del poema
  def devolverPoema(self):
    lineas = []
    for n in self.estructura:
      lineas.append("")
    for n in self.pilaPoema:
      actual = lineas[n.getNumVerso()]
      lineas[n.getNumVerso()] = juntaStrings(actual, n.getResultado())
    return lineas

  #Saca por pantalla el poema
  def printPoema(self):
    lineas = self.devolverPoema()
    for i in range(0,len(lineas)):
      if lineas[i] != "":
        print(i, lineas[i], " <--- Metrica del verso:", metricaVerso(lineas[i]), self.estructura[i][1], "Rima:", self.diccionarioRimas[self.estructura[i][1]])
      else:
        print(i)

  #Comprobaremos cual es el siguiente verso a generar
  def getVersoAGenerar(self):
    for n in range(0,len(self.estructura)):
      verso = self.getVerso(n)
      mVerso = metricaVerso(verso)
      if mVerso != self.estructura[n][0]:
        return n
    return None

  #Actualizamos el diccionario de las rimas
  def actualizarDiccionarioRimas(self):
    for n in self.diccionarioRimas:
      contAux = 0
      primerVerso = ""
      for i in range(0, len(self.estructura)):
        if self.estructura[i][1] == n: 
          contAux += 1
          if contAux == 1: #Guardaremos la ultima palabra de este verso para guardar la rima
            primerVerso = self.getVerso(i)
            mPrimerVerso = metricaVerso(primerVerso)
            if mPrimerVerso != self.estructura[i][0]: #Si este verso no está completo no será válido (tiene que ser metrica igual al objetivo descrito en la estructura)
              primerVerso = ""
          if contAux == 2 and primerVerso != "": #El segundo verso es el que definirá la rima de ese tipo de rima para todo el poema
            verso = self.getVerso(i)
            mVerso = metricaVerso(verso)
            if mVerso == self.estructura[i][0]: #Si este verso está completo (tiene que ser metrica igual al objetivo descrito en la estructura)
              self.diccionarioRimas[n] = self.getRimaVersos(primerVerso, verso)

  #Método principal que utiliza todos los anteriores para generar el poema
  def generarPoema(self):
    self.TOP_P = CONST_TOP_P
    self.MIN_TOP_P = CONST_MIN_TOP_P
    self.contadorPasosAtras = 0
    self.pilaPoema = []
    self.pedirEstructura()#Pedimos estructura
    if len(self.estructura) > 0:
      self.pedirEntrada() #Pedimos entrada
      if len(self.entrada) > 0:
        n = 0
        primerNodo = NodoGeneracion(self.entrada, 0, False, True) #Generamos el primer nodo
        primerNodo.setResultado(self.entrada)
        while n != None: #Iremos verso por verso
          print("GENERANDO VERSO", n)
          print("Estado de diccionario de rimas:")
          print(self.diccionarioRimas)
          if len(self.pilaPoema) == 0:
            print("METEMOS NODO INICIAL")
            primerNodo.setNumGeneraciones(0)
            self.pilaPoema.append(primerNodo)
          print("Estado de la pila del poema: ")
          [print(i, self.pilaPoema[i].getResultado(), "-->numVerso:",self.pilaPoema[i].getNumVerso()) for i in range(0, len(self.pilaPoema))]
          
          print("Estado poema: ")
          self.printPoema()
          if self.estructura[n][0] == 0: #Si es una linea en blanco (salto de linea)
            nuevoNodo = NodoGeneracion("", n, False, False)
            n+=1
          else:
            cortaMitad = CORTA_MITAD #Aqui se cambia el corta mitad
            v = self.getVerso(n)
            if v != "": #Un verso que ya tiene texto no se puede cortar por la mitad
              cortaMitad = False
            nuevoNodo, resultado = self.siguientePaso(self.getVersos(), n, cortaMitad)
            if resultado == 2:
              #Significa que cogeremos solamente la mitad del verso en cuanto a métrica y probaremos
              print("CORTAMOS POR LA MITAD:")
              textoNodo = nuevoNodo.getResultado()
              metricaMitad = math.floor(metricaVerso(textoNodo)/2)
              textoMitad, metricaResultante = getTextoMetrica(textoNodo, metricaMitad)
              print("-", textoNodo)
              print("-", textoMitad)
              nodoMitad = NodoGeneracion(nuevoNodo.getTextoEntrada(), n, False, False)
              nodoMitad.setResultado(textoMitad)
              self.pilaPoema.append(nodoMitad)
              self.actualizarDiccionarioRimas()
            elif resultado != None and nuevoNodo != None:
              #Si conseguimos un resultado            
              self.pilaPoema.append(nuevoNodo)
              self.actualizarDiccionarioRimas()              
            else:
              #Si no conseguimos un resultado, volveremos hacia atrás
              resultado = None
              while resultado == None:
                
                if len(self.pilaPoema) <= 1:
                  resultado = ""
                else:
                  print("Estado de la pila del poema: ")
                  [print(i, self.pilaPoema[i].getResultado(), "-->numVerso:",self.pilaPoema[i].getNumVerso()) for i in range(0, len(self.pilaPoema))] 
                  if len(self.pilaPoema) != 0:
                    resultado = self.anteriorPaso()
                    self.contadorPasosAtras += 1
                    if resultado == None or resultado[1] == 2: #No permitimos que al volver hacia atrás se partan los versos por la mitad
                      self.pilaPoema.pop() #Borramos el ultimo nodo ya que es inservible
                      resultado = None
                    else: #Si se ha conseguido un resultado correcto
                      self.actualizarDiccionarioRimas()
          n = self.getVersoAGenerar()
                
          print("------------------------------------------------------------------------------------")
        lineas = self.devolverPoema()
        print("RESULTADO: ")
        self.printPoema()
        
        return lineas
        
        
    return None


In [23]:
generadorPoema = GeneradorPoema()
texto = generadorPoema.generarPoema()

Primero de todo, deberá introducir una estructura, se mostrará el formato con un ejemplo: 
-     6a/6b/ /7a/7b/ /5-/6a
cada número seguido de una letra es un verso, los espacios significan cambio de estrofa y el guión (-) significa que ese verso no tiene rima
el mínimo silábico es 2 y el máximo 14
Introduzca una estructura: 11a/11b/11b/11a
['11a', '11b', '11b', '11a']
DiccionarioRimas: {'a': '', 'b': ''}
[(11, 'a'), (11, 'b'), (11, 'b'), (11, 'a')]
Ahora debe introducir una entrada a partir de la cual generar el resto del poema
Introduzca una entrada: Betis
GENERANDO VERSO 0
Estado de diccionario de rimas:
{'a': '', 'b': ''}
METEMOS NODO INICIAL
Estado de la pila del poema: 
0 Betis -->numVerso: 0
Estado poema: 
0 Betis  <--- Metrica del verso: 2 a Rima: 
1
2
3
Siguiente paso
GENERACION: 1
posible texto generado y Barcelona por un lado -->mAux: 11 -->mObj: 11
posible texto generado 1 - Valencia 0 Valencia -->mAux: 11 -->mObj: 11
posible texto generado y Las Palmas sellan el triunfo -->